In [32]:
import torch
from torch import nn
from Unet import UNet
import torch.nn.functional  as F
from torchvision.transforms import Compose
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset,DataLoader
from utils import *

In [ ]:
#数据路劲，每个类一个文件夹的形式方便使用
path = '/kaggle/input/cat-dataset'

In [ ]:
#大小看速度要求
pipe = Compose([transforms.Resize([224,224]),
               transforms.ToTensor(),
               ])

dataset = ImageFolder(path,
                      transform=pipe,
                      )
dataloader = DataLoader(dataset,
                        batch_size=32,#看设备需求
                        shuffle=True,
                        pin_memory=True)

# CT

In [39]:
#这些参数可调节
s_0=1 
s_1= 100

u_0=torch.tensor(0.99)   #ema初始化系数
k=0          
N=100      #时间分割步数
K= 1000    #训练次数  

def N_(k):
    a =torch.floor( torch.sqrt( k/K *(  (s_1 +1)**2  -  s_0**2   )    +  s_0**2)     -   1)
    return 1+a
def mu(k):
    return torch.exp(s_0 * torch.log(u_0)/N(k))
lam=1  #这个是样本损失，也可以做成函数
T=80   #论文设置
phi = 7  
eps= 0.002
i=torch.arange(1,N+1)
t =( eps**(1.0/phi)  +  (i-1)/(N-1)*(T**(1.0/phi) -  eps**(1.0/phi) )  )**(phi)
t[0]=eps

In [ ]:
online_network = UNet(3,3)                 
target_network = UNet(3,3)  
target_network.load_state_dict(online_network.state_dict())

In [ ]:
optimizer = torch.optim.Adam(online_network.parameters(),lr=0.001,weight_decay=0.99)
learn_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=10)

In [ ]:
while k < K:
    print(f'the {k} of all {K}')
    for data in dataloader:
        optimizer.zero_grad()
        image,_ =data
        bs = image.shape[0]
        n = torch.randint(low=1,high=N_(k)+1,size=(bs,1,1,1))
        shape = image.shape
        z = torch.rand(*shape)
        output = online_network(image+ t[n+1]*z,t[n+1])
        with torch.no_grad():
            output_min = target_network(image+ t[n]*z,t[n])
            
        loss = torch.sum((output - output_min.detach())**2)
        loss.backward()
        optimizer.step()
        m = mu(k)
        update(target_network,online_network,m)
        learn_scheduler.step()
        k+=1
        print(f'the iter is {k}')

# CD

In [18]:

#可调节参数，同上
N=100
T=80
phi = 7
eps= 0.002
i=torch.arange(1,N+1)
t =( eps**(1.0/phi)  +  (i-1)/(N-1)*(T**(1.0/phi) -  eps**(1.0/phi) )  )**(phi)
t[0]=eps
mu=0.99

In [3]:
online_network = UNet(3,3)
target_network = UNet(3,3)
target_network.load_state_dict(online_network.state_dict())

<All keys matched successfully>

In [12]:
#可以按照自己的喜好替换成任意训练好的Unet
score_network = UNet(3,3)
score_network.eval()

In [ ]:
#同CT
pipe = Compose([transforms.Resize([224,224]),
               transforms.ToTensor(),
               ])
dataset = ImageFolder(path,
                      transform=pipe,)
dataloader = DataLoader(dataset,
                        batch_size=4,
                        shuffle=True,
                        pin_memory=True)

In [24]:
optimizer = torch.optim.Adam(online_network.parameters(),lr=0.001,weight_decay=0.99)
learn_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=10)

In [ ]:
epoches = 12
for epoch in range(epoches):
    
    for data in dataloader:
        optimizer.zero_grad()
        x,_ = data
        z= torch.rand(*x.shape)
        
        bs = x.shape[0]
        n = torch.randint(low=1,high=N,size=(bs,1,1,1))
        
        x_n_post = x+z * t[n+1]
        x_n_pre = x_n_post - (t[n]-t[n+1]) *score_network(x_n_post,t[n+1])
        
        output = online_network(x_n_post,t[n+1])
        with torch.no_grad():
            
            output_min = target_network(x_n_pre,t[n])
        loss = torch.sum((output-output_min)**2)
        loss.backward()
        optimizer.step()
        update(target_network,online_network,mu)
        learn_scheduler.step()
        print('finish one iter')

# Multistep Consistency Sampling

In [19]:
n_t=torch.arange(1,N)

In [29]:
tao,_ = t[n_t].sort(descending=True)

In [30]:
x_T = torch.rand(1,3,224,224)
x= target_network(x,T)
for i in range(N):
    z =torch.rand(*x_T.shape)
    x_tao_n = x + torch.sqrt(tao[i]**2-eps**2)* z
    x = target_network(x_tao_n,t[n])
    print('finish one iter')


tensor([75.6912, 71.5828, 67.6671, 63.9365, 60.3835, 57.0012, 53.7825, 50.7209,
        47.8100, 45.0434])